In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(30)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

directory_out = './file/3.part_2'

if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

# = = = = = = = = = = = = = = =

directory_in = './file/2.part'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
for file in list_file:

    # = = = = = = = = = = = = = = =

    list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                           header=0,
                                                                           dtype=str).fillna('')

    df_input['No'] = df_input['No'].astype(int)

    count_remain = len(df_input)

    print(f'总数量：{count_remain}')
    print()

    # = = = = = = = = = = = = = = =

    def crawler(crawler_tuple):
        global list_dict_correct, list_series_error, count_remain

        crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

        # = = = = = = = = = = = = = = =

        try:
            count_retry = 0
            while True:
                count_retry += 1

                try:
                    url_request = crawler_series['Url']

                    resp = requests.get(url_request,
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(5, 15))

                    if resp.status_code == 200:
                        soup = BeautifulSoup(resp.text, 'lxml')
                        html = etree.HTML(str(soup))

                        group, type_ = html.xpath('//select[@id="partgroup_partsearch_007"]/option[@selected]/text()')[0].strip(), html.xpath('//select[@id="parttype_partsearch_007"]/option[@selected]/text()')[0].strip()
                        break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            list_dict_correct.append({'No': crawler_series['No'],
                                      'Group': group,
                                      'Type': type_,
                                      'Manufacturer': crawler_series['Manufacturer'],
                                      'Part Number': crawler_series['Part Number'],
                                      'OEM': crawler_series['OEM'],
                                      'Picture': crawler_series['Picture'],
                                      'Url': f'''https://www.rockauto.com/en/parts/{crawler_series['Manufacturer']},{crawler_series['Part Number']},{type_},{crawler_series['Type Code']}''',
                                      'Json_Src': crawler_series['Json_Src'],
                                      'Info': crawler_series['Info'],
                                      'Json_Note 1': crawler_series['Json_Note 1'],
                                      'Json_Note 2': crawler_series['Json_Note 2'],
                                      'Choose': crawler_series['Choose'],
                                      'Json_Price': crawler_series['Json_Price'],
                                      'Type Code': crawler_series['Type Code'],
                                      'Part Code': crawler_series['Part Code'],
                                      'JOIN_MPNTCPC': crawler_series['JOIN_MPNTCPC']})

            # = = = = = = = = = = = = = = =

            crawler_status = True
        except:
            crawler_series['Request_Url'] = url_request

            with semaphore_error:
                list_series_error.append(crawler_series)

        # = = = = = = = = = = = = = = =

        if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and count_remain == 1 or count_remain % 300 == 0 and count_remain != 0:
            clear_output()
        print(f'''[{crawler_status}] - [请求{count_retry}次] - [剩余{count_remain - 1}条] - [{file.removesuffix('.xlsx')}（{(list_file.index(file) + 1) / len(list_file) * 100:.2f}%）] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_request}\n''')

        with semaphore_remain:
            count_remain -= 1

    # = = = = = = = = = = = = = = =

    pool.map(crawler, df_input.iterrows())

    print('输出ing...')
    print()
    if list_dict_correct:
        df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                                 ascending=[True],
                                                                 ignore_index=True)
        df_correct.to_excel(f'''{directory_out}/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if list_series_error:
        df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                               ascending=[True],
                                                               ignore_index=True)
        df_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-part_2_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = [file for file in sorted(os.listdir(directory_out)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_out}/{file}')]
for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'{directory_out}/{file}',
              f'''{directory_out}/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：10

[True] - [请求6次] - [剩余9条] - [part_number（100.00%）] - [2024/10/29 08:29:40] - 2. https://www.rockauto.com/en/parts/FOUR SEASONS,33188,Lennon,6972

[True] - [请求7次] - [剩余8条] - [part_number（100.00%）] - [2024/10/29 08:29:40] - 1. https://www.rockauto.com/en/parts/FOUR SEASONS,33072,Lennon,6972

[True] - [请求10次] - [剩余7条] - [part_number（100.00%）] - [2024/10/29 08:29:40] - 3. https://www.rockauto.com/en/parts/FOUR SEASONS,56790,Lennon,6972

[True] - [请求11次] - [剩余6条] - [part_number（100.00%）] - [2024/10/29 08:29:42] - 10. https://www.rockauto.com/en/parts/FOUR SEASONS,83052,Lennon,6972

[True] - [请求15次] - [剩余5条] - [part_number（100.00%）] - [2024/10/29 08:29:43] - 4. https://www.rockauto.com/en/parts/FOUR SEASONS,33094,Lennon,6972

[True] - [请求12次] - [剩余4条] - [part_number（100.00%）] - [2024/10/29 08:29:43] - 7. https://www.rockauto.com/en/parts/FOUR SEASONS,33098,Lennon,6972

[True] - [请求22次] - [剩余3条] - [part_number（100.00%）] - [2024/10/29 08:29:45] - 8. https://www.rockauto.com/en/parts/FOU

Progress: 100%|██████████████████████████████| 1/1 [00:00<00:00, 2468.69it/s]

Done ~
